In [1]:
import requests

In [7]:
#Define Request Payload
infer_clip_payload = {
    #Images can be provided as urls or as base64 encoded strings
    "image": {
        "type": "url",
        "value": ,
    },
}

# Define inference server url (localhost:9001, infer.roboflow.com, etc.)
base_url = "https://infer.roboflow.com"

# Define your Roboflow API Key
api_key = "pUmnI6Vv3mdDdmDiEtqz"

res = requests.post(
    f"{base_url}/clip/embed_image?api_key={api_key}",
    json=infer_clip_payload,
)

embeddings = res.json()

print(embeddings)

{'frame_id': None, 'time': 6.156852136999987, 'embeddings': [[-0.21139085292816162, -0.2235783338546753, -0.6934599876403809, -0.11402460932731628, -0.38369375467300415, 0.13590869307518005, 0.5036612749099731, -0.04922281205654144, 0.5610694885253906, -0.41703909635543823, 0.5099771022796631, 1.2224793434143066, 0.5197300910949707, -0.8962376713752747, 0.16710372269153595, -0.627465009689331, -0.09061037749052048, 0.08778490126132965, -0.3021768629550934, 0.0830419659614563, 0.12344050407409668, 0.3510016202926636, -0.15540564060211182, -0.3264020085334778, -0.07311730086803436, 0.13427433371543884, -0.09284193068742752, -0.13276931643486023, 0.24581317603588104, 0.19951081275939941, 0.4841715693473816, 0.38477176427841187, -0.3303312063217163, 0.07641328126192093, 0.4147939085960388, -0.49149858951568604, -0.09053172916173935, 0.3964062035083771, -0.04657094180583954, 0.0669977217912674, -0.15415892004966736, -0.06869076937437057, -0.15933199226856232, 0.25341591238975525, 0.12370993

In [12]:
len(embeddings['embeddings'][0])

512

In [14]:
import io
import os
import base64
import numpy as np
from PIL import Image
import pymupdf
import fitz

In [18]:
def encode_image(image_path):
    ''' Getting the base64 string '''
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [23]:
def get_img_embed(image_path):
    encoded_val = encode_image(image_path)
    infer_clip_payload = {
        "image": {
            "type": "base64",
            "value": f"{encoded_val}",
        },
    }

    # Define inference server url (localhost:9001, infer.roboflow.com, etc.)
    base_url = "https://infer.roboflow.com"

    # Define your Roboflow API Key
    api_key = "pUmnI6Vv3mdDdmDiEtqz"

    res = requests.post(
        f"{base_url}/clip/embed_image?api_key={api_key}",
        json=infer_clip_payload,
    )

    embeddings = res.json()

    if "embeddings" in embeddings:
        return embeddings['embeddings'][0]
    else:
        return None


In [24]:
base_path = "F:\\psg\\bosch_hackathon\\input_data\\"

In [25]:
val = get_img_embed(base_path + "engine.jpeg")

In [28]:
val_1 = get_img_embed(base_path + "engine_1.jpg")
val_2 = get_img_embed(base_path + "cat.jpg")

In [30]:
from sentence_transformers.util import cos_sim


In [31]:
cos_sim(val, val_1)

tensor([[0.7303]])

In [32]:
cos_sim(val, val_2)

tensor([[0.4657]])

In [33]:
cos_sim(val, val)

tensor([[1.0000]])

In [34]:
import os
from operator import itemgetter
from langchain.load import dumps, loads
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
import numpy as np
import time
from sentence_transformers.util import cos_sim
from langchain_core.documents.base import Document
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
import qdrant_client
from langchain.chains import ConversationalRetrievalChain
import pandas as pd
import json
from qdrant_client import models, QdrantClient
import cohere
import numpy as np
from qdrant_client.http.models import Batch
import concurrent.futures
from io import BytesIO
from pdf2image import convert_from_path
import streamlit.components.v1 as components
import base64
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
from PIL import Image
import torch
import io

In [46]:
QDRANT_URL = "https://8803fa99-7551-4f88-84c3-e134c9bed5de.us-east4-0.gcp.cloud.qdrant.io:6333"
QDRANT_API_KEY = "EFeN_UhdmAlDNYZHqJBUbZ88Nt7N0MkmvWLgM5Hs4ogNvExLMwNwdQ"
QDRANT_COLLECTION_NAME = "owners_manual_chunks"


In [70]:

def return_images_context(image_ids):
    text_to_image_ids = dict()

    qdrant_client = QdrantClient(
        QDRANT_URL,
        prefer_grpc=True,
        api_key=QDRANT_API_KEY,
    )

    should_filters = list()

    for i in image_ids:
        print(i)
        should_filters.append(
            models.FieldCondition(
                    key="metadata.image_ids",
                    match=models.MatchValue(value=i),
                )
        )

    must_filters=[models.FieldCondition(key="metadata.chunk_type", match=models.MatchValue(value="Image"))]

    for i in qdrant_client.scroll(collection_name=f"{QDRANT_COLLECTION_NAME}", scroll_filter=models.Filter(should=should_filters, must=must_filters),limit=100)[0]:
        print("hi there")
        text_to_image_ids[i.payload['page_content']] = i.payload['metadata']['image_ids']

    return text_to_image_ids

In [74]:

def get_image_context_from_QDrant(image_vector):
        qdrant_client = QdrantClient(
            "https://35ebdc7d-ec99-4ebd-896c-ff5705cf369b.us-east4-0.gcp.cloud.qdrant.io:6333",
            prefer_grpc=True,
            api_key="9dKJsKOYwT0vGlWPrZXBSIlbUzvRdJ1XkM0_floo8FmYCOHX_Y0y-Q",
        )

        search_result = qdrant_client.search(
            collection_name="owners_manual_images_roboflow",
            query_vector=image_vector,
            limit = 1
        )

        payloads = [hit.payload for hit in search_result]
        print(payloads)
        image_id = payloads[0]['metadata']['image_id']
        print(image_id)
        image_context = return_images_context([image_id])
        print(image_context)
        image_summary = list(image_context.keys())[0]
        print(image_summary)
        return image_summary

In [75]:
def get_image_summary_roboflow(image_path):
    import requests
    encoded_val = encode_image(image_path)
    infer_clip_payload = {
        "image": {
            "type": "base64",
            "value": f"{encoded_val}",
        },
    }

    # Define inference server url (localhost:9001, infer.roboflow.com, etc.)
    base_url = "https://infer.roboflow.com"

    # Define your Roboflow API Key
    api_key = "pUmnI6Vv3mdDdmDiEtqz"

    res = requests.post(
        f"{base_url}/clip/embed_image?api_key={api_key}",
        json=infer_clip_payload,
    )

    embeddings = res.json()

    if "embeddings" in embeddings:
        print(embeddings['embeddings'])
        return get_image_context_from_QDrant(embeddings['embeddings'][0])
    else:
        return None

In [76]:
get_image_summary_roboflow(base_path+"engine.jpeg")

[[-0.21139085292816162, -0.2235783338546753, -0.6934599876403809, -0.11402460932731628, -0.38369375467300415, 0.13590869307518005, 0.5036612749099731, -0.04922281205654144, 0.5610694885253906, -0.41703909635543823, 0.5099771022796631, 1.2224793434143066, 0.5197300910949707, -0.8962376713752747, 0.16710372269153595, -0.627465009689331, -0.09061037749052048, 0.08778490126132965, -0.3021768629550934, 0.0830419659614563, 0.12344050407409668, 0.3510016202926636, -0.15540564060211182, -0.3264020085334778, -0.07311730086803436, 0.13427433371543884, -0.09284193068742752, -0.13276931643486023, 0.24581317603588104, 0.19951081275939941, 0.4841715693473816, 0.38477176427841187, -0.3303312063217163, 0.07641328126192093, 0.4147939085960388, -0.49149858951568604, -0.09053172916173935, 0.3964062035083771, -0.04657094180583954, 0.0669977217912674, -0.15415892004966736, -0.06869076937437057, -0.15933199226856232, 0.25341591238975525, 0.12370993196964264, -0.03892979025840759, -0.08103333413600922, 0.116

'This Image is extracted from manual of Hyundai Exter. The image depicts a diagram of the engine compartment of a vehicle. It labels various components including the engine coolant reservoir, engine oil filler cap, brake/clutch fluid reservoir, air cleaner, fuse box, battery, windshield washer fluid reservoir, radiator cap, engine oil dipstick, and CNG gaseous fuel filter. The actual engine compartment in the vehicle may differ from the illustration. The image highlights the essential parts of the engine bay, providing a visual guide for understanding the layout and location of key components. \n'

In [67]:
return_images_context(['image_hyundai_exter_381_1'])

{}